Входные данные:



3 csv файла - каждый файл -- это координаты gps, где человек находился какое-|то время


##### Информация
##### latitude - Широта
##### longitude - Долгота
Timestamp
    - YYYY = year
    - MM = month of year
    - dd = day of month
    - HH=hourofday
    - mm = minute of hour
    - Z = timezone offset
Duration - продолжительность пребывания человека в локации в мс

## Задание 
- Посещали пользователи одинаковые места? Знают ли они друг друга?
- Ввести метрику социальной активности - она должна учитывать количество мест, которое посетил пользователь, как далеко они друг от друга и как долго человек оставался там - метрика должна позволять сравнивать пользователей и находить наиболее активных


In [1]:
import numpy as np
import pandas as pd
import folium
import time
import math

In [37]:
person1 = pd.read_csv("person.1.csv", sep = ";")
person2 = pd.read_csv("person.2.csv", sep = ";")
person3 = pd.read_csv("person.3.csv", sep = ";")

In [38]:
!pip install geopy

In [39]:
!pip install folium

In [40]:
latitude1 = person1["latitude"]
longitude1 = person1["longitude"]
latitude2 = person2["latitude"]
longitude2 = person2["longitude"]
latitude3 = person3["latitude"]
longitude3 = person3["longitude"]

In [41]:
from geographiclib.geodesic import Geodesic
geod = Geodesic.WGS84

param1 = []
for i in range(len(latitude1)-1):
    param1.append(geod.Inverse(float(latitude1[latitude1.index.isin([i])]), 
                                     float(longitude1[longitude1.index.isin([i])]),
                                     float(latitude1[latitude1.index.isin([i+1])]), 
                                     float(longitude1[longitude1.index.isin([i+1])])))

param2 = []
for i in range(len(latitude2)-1):
    param2.append(geod.Inverse(float(latitude2[latitude2.index.isin([i])]), 
                                     float(longitude2[longitude2.index.isin([i])]),
                                     float(latitude2[latitude2.index.isin([i+1])]), 
                                     float(longitude2[longitude2.index.isin([i+1])])))

param3 = []
for i in range(len(latitude3)-1):
    param3.append(geod.Inverse(float(latitude3[latitude3.index.isin([i])]), 
                                     float(longitude3[longitude3.index.isin([i])]),
                                     float(latitude3[latitude3.index.isin([i+1])]), 
                                     float(longitude3[longitude3.index.isin([i+1])])))    

# Создадим списки из расстояний от i геопозиции j человека из выборки до его i+1 геопозиции. 
# Так как геопозиции даны последовательно во времени, то достаточно посчитать distance[i, i+1] и не считать попарные расстояния
# между каждой геопозицией.

dist1 = []
for elem in param1:
    dist1.append(elem["s12"]/1000)
total_dist1 = sum(dist1)
print(total_dist1, "km", sep='')

dist2 = []
for elem in param2:
    dist2.append(elem["s12"]/1000)
total_dist2 = sum(dist2)
print(total_dist2, "km", sep='')

dist3 = []
for elem in param3:
    dist3.append(elem["s12"]/1000)
total_dist3 = sum(dist3)
print(total_dist3, "km", sep='')

19006.560444243754km
2632.0522529050627km
4716.878815467023km


In [7]:
!pip install reverse_geocoder

In [8]:
# С помощью обратного геокодера координат сможем определить места, которые посещали люди из выборки
import reverse_geocoder as rg

loc_list1 = []
loc_list2 = []
loc_list3 = []

def reverseGeocoder(coordinates):        
    result = rg.search(coordinates)      
    return result

for coord in range(len(latitude1)):
    coordinates =  (float(latitude1[latitude1.index.isin([coord])]), float(longitude1[longitude1.index.isin([coord])]))
    loc_list1.append(reverseGeocoder(coordinates))

for coord in range(len(latitude2)):
    coordinates =  (float(latitude2[latitude2.index.isin([coord])]), float(longitude2[longitude2.index.isin([coord])]))
    loc_list2.append(reverseGeocoder(coordinates))

for coord in range(len(latitude3)):
    coordinates =  (float(latitude3[latitude3.index.isin([coord])]), float(longitude3[longitude3.index.isin([coord])]))
    loc_list3.append(reverseGeocoder(coordinates))
    


Loading formatted geocoded file...


In [42]:
# Cоздадим списки unique_list(i) из уникальных мест, которые посетили люди 
# и списки loc_list(i)_mod из всех мест, посещенных каждым по порядку
# (более удобная для работы форма списка loc_list(i), так как в ней только название локации в виде строки)

loc_list1_mod = []
unique_list1 = []
for i in range(len(loc_list1)):
    loc_list1_mod.append(loc_list1[i][0]['name'])
    if loc_list1[i][0]['name'] not in unique_list1:
        unique_list1.append(loc_list1[i][0]['name'])
        
loc_list2_mod = []
unique_list2 = []
for i in range(len(loc_list2)):
    loc_list2_mod.append(loc_list2[i][0]['name'])
    if loc_list2[i][0]['name'] not in unique_list2:
        unique_list2.append(loc_list2[i][0]['name'])

loc_list3_mod = []
unique_list3 = []
for i in range(len(loc_list3)):
    loc_list3_mod.append(loc_list3[i][0]['name'])
    if loc_list3[i][0]['name'] not in unique_list3:
        unique_list3.append(loc_list3[i][0]['name'])

In [43]:
# Создадим списки для каждого человека из выборки, в которые запишем сколько раз j человек из выборки бывал в данном месте 
# к тому моменту когда мы рассматриваем его i геопозицию.
# Я считаю, что разумно сделать предположение, что чем чаще человек посещает какое-то место, тем ниже вероятность встретить 
# в нем новых людей и обзавесьтись новыми социальными связями. Поэтому при оценке вклада этого места в социальную активность 
# надо учитывать его каждый раз все с меньшим весом.

number1 = []
for i in range(len(loc_list1)):
    number1.append(loc_list1_mod[:i].count(loc_list1[i][0]['name']) + 1)

number2 = []
for i in range(len(loc_list2)):
    number2.append(loc_list2_mod[:i].count(loc_list2[i][0]['name']) + 1)
    
number3 = []
for i in range(len(loc_list3)):
    number3.append(loc_list3_mod[:i].count(loc_list3[i][0]['name']) + 1)

# Добавим новый признак в датафреймы

person1["number_of_visits"] = number1
person2["number_of_visits"] = number2
person3["number_of_visits"] = number3

In [44]:
# Переведем время из милисекунд в секунды

dur_min1 = person1["duration(ms)"]/60000
dur_min2 = person2["duration(ms)"]/60000
dur_min3 = person3["duration(ms)"]/60000

In [45]:
# Добавим признак, измеренный в секундах в датафреймы

person1["duration(min)"] = dur_min1
person2["duration(min)"] = dur_min2
person3["duration(min)"] = dur_min3

# Уберем признак в милисекундах
person1 = person1.drop("duration(ms)", axis = 'columns')
person2 = person2.drop("duration(ms)", axis = 'columns')
person3 = person3.drop("duration(ms)", axis = 'columns')

In [46]:
person1["duration(min)"].sum(), person2["duration(min)"].sum(), person3["duration(min)"].sum() 

(95350.94841666665, 114396.15861666667, 55884.20020000001)

In [47]:
# Перекодируем время в формат, который используется в модуле datatime
# Возможно пригодится для дальнейших задач
time1 = []
for i in person1["start_time(YYYYMMddHHmmZ)"]:
    time1.append(str(i)[:4] + "-" + str(i)[4:6] + "-" + str(i)[6:8] + " " + str(i)[8:10] + ":" + str(i)[10:12] + ":00")

time2 = []
for i in person2["start_time(YYYYMMddHHmmZ)"]:
    time2.append(str(i)[:4] + "-" + str(i)[4:6] + "-" + str(i)[6:8] + " " + str(i)[8:10] + ":" + str(i)[10:12] + ":00")
    
time3 = []
for i in person3["start_time(YYYYMMddHHmmZ)"]:
    time3.append(str(i)[:4] + "-" + str(i)[4:6] + "-" + str(i)[6:8] + " " + str(i)[8:10] + ":" + str(i)[10:12] + ":00")



In [48]:
person1["start_time(YYYYMMddHHmmZ)"] = time1
person2["start_time(YYYYMMddHHmmZ)"] = time2
person3["start_time(YYYYMMddHHmmZ)"] = time3

In [49]:
# Добавим в датафреймы столбец расстояний от одного места до другого (в км). Так как gps координаты даны последовательно, 
# то достаточно посчитать расстояние от gps позиции i до gps позиции i+1, где i = 0,..,(число геопозиций -1). 
# distance_to_place(0) = 0

person1["distance_to_place"] = [0] + dist1
person2["distance_to_place"] = [0] + dist2
person3["distance_to_place"] = [0] + dist3

# Добавим такой параметр в таблицу, как приоритетность места. Этот параметр будет от 0 до 3.
# Я предполагаю, что 5-минутные пребывания в каком-то месте можно считать случайными, так как они редко приводят к какому-то 
# социальному взаимодействию. Поэтому таке контакты можно исключить из рассмотрения путем введения нулевого коэффициента на них
# (Порог для случайного взаимодействия можно устанавливать разный. Зависит от задачи. Аналогичное справедливо и для других
# порогов, которые будут представлены ниже).
# Взаимодействия от 5 до 30 минут с какой-то локацией приведут к взаимодействию с большей вероятностью, поэтому для них 
# приоритетность 1. Места, в которых люди могут задерживаться от 30 до 60 минут это обычно рестораны, кафе, кофейни и т. д.,
# а в них люди обычно не ходят одни и взаимодействуют со знакомыми и друзьями. Для таких мест коэффициент 2. Места, в которых
# люди задерживаются дольше часа это обычно самые важные места: работа, учеба, спортивные объекты, где невозможно представить
# человека без взаимодействия. Поэтому коэффициент 3.

# Изначально инициализируем 0

person1["place_priority"] = np.zeros(len(person1['duration(min)']))
person2["place_priority"] = np.zeros(len(person2['duration(min)']))
person3["place_priority"] = np.zeros(len(person3['duration(min)']))

In [50]:
for i in range(len(person1['duration(min)'])):
    if person1['duration(min)'][i] > 60:
        person1["place_priority"][i] = 3
    elif (person1['duration(min)'][i] > 30 and person1['duration(min)'][i] < 60):
        person1["place_priority"][i] = 2
    elif (person1['duration(min)'][i] > 5 and person1['duration(min)'][i] < 30):
        person1["place_priority"][i] = 1
    else:
        pass

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [51]:
for i in range(len(person2['duration(min)'])):
    if person2['duration(min)'][i] > 60:
        person2["place_priority"][i] = 3
    elif (person2['duration(min)'][i] > 30 and person2['duration(min)'][i] < 60):
        person2["place_priority"][i] = 2
    elif (person2['duration(min)'][i] > 5 and person2['duration(min)'][i] < 30):
        person2["place_priority"][i] = 1
    else:
        pass

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [52]:
for i in range(len(person3['duration(min)'])):
    if person3['duration(min)'][i] > 60:
        person3["place_priority"][i] = 3
    elif (person3['duration(min)'][i] > 30 and person3['duration(min)'][i] < 60):
        person3["place_priority"][i] = 2
    elif (person3['duration(min)'][i] > 5 and person3['duration(min)'][i] < 30):
        person3["place_priority"][i] = 1
    else:
        pass

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Составим индекс с помощью которого будем оценивать социальную активность людей из списка.
Мне показалось, что его можноо выстроить следующим образом:
activity(i) = 
SUM{log(duration(min)(i) * log(distance_to_place)(i) * place_priority(i) * (alpha)^(number_of_visits)(i)} / число gps отметок 
alpha = 0.995 (значение в 0.995 показалось оптимальным, но может быть любое число меньше 1, но лучше чтоб оно было близко к 1)
i = 1, ... , (число геолокаций)

Объяснение вида индекса:
    1) Мне показалось оптимальным, чтоб индекс был безразмерным, так как время величина безразмерная

    2) Я взял логарифмы, так как расстояние в километрах может быть от 0 до ~20 000 км (половина экватора), а время как уже было сказано безгранично. Таким образом, мне необходима функция, которая сглаживает такой разброс величин и растет убывающими темпами, потому что задержался человек на день в каком-то месте особо качественно не сильно различается, а 20 минут и 40 минут имеют значение.

    3) Я взял произведение логарифмов, так как мне необходимо, чтоб максимально далекие объекты от текущей позиции влияли и время, проведенное в каком-то месте положительно влияло на индекс. Помимо этого необходимо учесть следующие моменты. Если человек проводит много времени и при этом перемещается до него недолго, то скорее всего это работа, учеба или место проживания. Все эти места это обычная среда, поэтому они не должны вносить большой вклад в индекс за счет большого проведенного времени. Произведение позволяет перемещениям до работы с помощью маленького логарифма расстояния быть небольшой величиной. Также поездка далеко и надолго должна вносить самый высокий вклад в индекс (туризм). Произведение позволяет это сделать.

    4) Суть приоритетность места была объяснена выше. Этот коэффициент искуственно добавляет важности походам на работу, учебу, которая урезается короткими расстояниями до них.

    5) Но вклад этих мест в социальную активность должен падать, так как они ограничивают человека в новых социальных взаимодействиях, поэтому я добавил так называемый "дисконт-фактор", который обесценивает каждый новый поход.
    
    6) Сумма i мест была взята так как каждое вновь посещенное место должно положительно влиять на индекс
    
    7) Деление на число геолокаций взято для того, чтобы людей, для которых есть данные для разного числа местоположений можно было сравнивать между собой. (по сути среднее)

Таким образом индекс учитывает:
    1) Число мест
    2) Расстояния между ними
    3) Время, проведенное в каждом из мест
    4) Разные объемы выборок

*distance_to_place(0) = 0

In [53]:
activity1 = []
for i in range(1, len(person1['duration(min)'])):
    activity1.append(math.log1p(person1['duration(min)'][i])*math.log1p(dist1[i-1])*person1['place_priority'][i]*0.995**person1["number_of_visits"][i])

activity2 = []
for i in range(1, len(person2['duration(min)'])):
    activity2.append(math.log1p(person2['duration(min)'][i])*math.log1p(dist2[i-1])*person2['place_priority'][i]*0.995**person2["number_of_visits"][i])

activity3 = []
for i in range(1, len(person3['duration(min)'])):
    activity3.append(math.log1p(person3['duration(min)'][i])*math.log1p(dist3[i-1])*person3['place_priority'][i]*0.995**person3["number_of_visits"][i])


In [54]:
person1['activity'] = [0] + activity1
person2['activity'] = [0] + activity2
person3['activity'] = [0] + activity3

In [55]:
act_index1 = sum(activity1)/len(person1['activity'])
act_index2 = sum(activity2)/len(person2['activity']) 
act_index3 = sum(activity3)/len(person3['activity'])

In [56]:
act_index1, act_index2, act_index3

(5.877893412993733, 2.108821546019175, 12.509025191838386)

На основе посчитанных значений индекса можно предположить, что самый социально активный это "person3"

In [57]:
from datetime import datetime, timedelta

time_period1 = []
for i in range(len(person1['start_time(YYYYMMddHHmmZ)'])):
    time_period1.append(
        [datetime.fromisoformat(str(person1['start_time(YYYYMMddHHmmZ)'][i])).ctime(),
        (datetime.fromisoformat(str(person1['start_time(YYYYMMddHHmmZ)'][i])) + 
                         timedelta(minutes = person1['duration(min)'][i], seconds = 0, microseconds = 0)).ctime()])

time_period2 = []
for i in range(len(person2['start_time(YYYYMMddHHmmZ)'])):
    time_period2.append(
        [datetime.fromisoformat(str(person2['start_time(YYYYMMddHHmmZ)'][i])).ctime(),
        (datetime.fromisoformat(str(person2['start_time(YYYYMMddHHmmZ)'][i])) + 
                         timedelta(minutes = person2['duration(min)'][i], seconds = 0, microseconds = 0)).ctime()])

time_period3 = []
for i in range(len(person3['start_time(YYYYMMddHHmmZ)'])):
    time_period3.append(
        [datetime.fromisoformat(str(person3['start_time(YYYYMMddHHmmZ)'][i])).ctime(),
        (datetime.fromisoformat(str(person3['start_time(YYYYMMddHHmmZ)'][i])) + 
                         timedelta(minutes = person3['duration(min)'][i], seconds = 0, microseconds = 0)).ctime()])

In [58]:
# Нарисуем карту мест, в которых были люди, представленные в выборке
map = folium.Map(location = [0, 0], 
                 zoom_start = 2) 

In [59]:
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(map)

for i in range(len(latitude1)):
    folium.CircleMarker(location = [float(latitude1[latitude1.index.isin([i])]), float(longitude1[longitude1.index.isin([i])])], 
                        tooltip = str(time_period1[i][0] + "  " + time_period1[i][1] + "  " + str(i)),
                        fill_color = "red",
                        tiles="Mapbox Bright",
                        radius = 9,
                        color = "gray",
                        
                        fill_opacity = 0.9).add_to(marker_cluster)

for i in range(len(latitude2)):
    folium.CircleMarker(location = [float(latitude2[latitude2.index.isin([i])]), float(longitude2[longitude2.index.isin([i])])], 
                        tooltip = str(time_period2[i][0] + "  " + time_period2[i][1] + "  " + str(i)),
                        fill_color =  'green',
                        tiles="Mapbox Bright",
                        radius = 9,
                        color = "gray",
                        fill_opacity = 0.9).add_to(marker_cluster)  

for i in range(len(latitude3)):
    folium.CircleMarker(location = [float(latitude3[latitude3.index.isin([i])]), float(longitude3[longitude3.index.isin([i])])], 
                        tooltip = str(time_period3[i][0] + "  " + time_period3[i][1] + "  " + str(i)),
                        fill_color = 'blue',
                        tiles="Mapbox Bright",
                        radius = 9,
                        color = "gray",
                        fill_opacity = 0.9).add_to(marker_cluster)    


In [60]:
map

Если предположить, что 2 человека знакомы если находились в одно время в одном месте, то можно считать что 1, 2 и 3 человек 
знакомы между собой. Приведу пример такого места:

In [61]:
person1.iloc[212:213]

,latitude,longitude,start_time(YYYYMMddHHmmZ),number_of_visits,duration(min),distance_to_place,place_priority,activity
212,51.216316,4.394118,2014-01-15 11:15:00,11,325.5777,0.398154,3.0,5.508029


In [62]:
person2.iloc[189:190]

,latitude,longitude,start_time(YYYYMMddHHmmZ),number_of_visits,duration(min),distance_to_place,place_priority,activity
189,51.21632,4.394156,2014-01-15 08:19:00,135,499.475633,0.198199,3.0,1.713811


In [63]:
person3.iloc[47:48]

,latitude,longitude,start_time(YYYYMMddHHmmZ),number_of_visits,duration(min),distance_to_place,place_priority,activity
47,51.216297,4.394103,2014-01-15 07:55:00,17,553.283933,45.571142,3.0,66.851817
